In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import glob

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [5]:
data_type = "test"
# month = "07"
# category = "잔액정보"

# local
root_path = '../data/open'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

drive_folder = f'{root_path}/{data_type}/5.잔액정보/'

In [7]:
# 잔액정보 (모든 월 통합) 통합 parquet
balance = pd.read_parquet(f'{drive_folder}test_잔액정보_통합.parquet')
balance

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,...,평잔_6M,평잔_일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M
0,201807,TEST_00000,4802,0,0,0,0,0,6412,0,...,3847,2933,384,0,0,0,0,0,0,0
1,201807,TEST_00001,1201,8628,5448,0,0,24299,1010,8156,...,42634,657,0,0,0,7979,0,1530,0,31707
2,201807,TEST_00002,3046,6144,0,0,0,0,4407,8033,...,22936,6253,0,0,0,12329,0,0,0,0
3,201807,TEST_00003,768,945,0,0,0,0,1107,674,...,1206,819,0,0,0,468,0,0,0,0
4,201807,TEST_00004,637,2558,0,0,0,0,1072,3690,...,2914,979,0,0,0,2414,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599996,201812,TEST_99996,191,0,0,0,0,0,260,0,...,275,137,0,0,0,0,0,0,0,0
599997,201812,TEST_99997,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599998,201812,TEST_99998,17916,2786,0,0,0,0,16911,2081,...,20549,18108,240,0,0,1830,0,0,0,0


In [5]:
# 메모리 사용량이 너무 많아서, int64를 int32로 바꾼다.
for col in balance.select_dtypes(include='int64').columns:
    balance[col] = balance[col].astype('int32')

In [7]:
# 출력 옵션 조정 (생략 없이 dtypes 포함 모든 열 보기)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 

In [9]:
# 🔎 결측치 확인
# print("🔍 [결측치 개수]")
# print("▶ balance:\n", balance.isnull().sum().sort_values(ascending=False))

- 연체일자_BOM만 결측치가 있는 것으로 확인했음.

In [12]:
# 연체일자_BOM는 거의 결측치
# 묻지도 않고 삭제하고 살펴보겠음.
balance.drop(columns=['연체일자_B0M'], inplace=True)

In [14]:
# 🔎 데이터 타입
# print("▶ balance dtypes\n", balance.dtypes)

In [16]:
# 🔎 빈 문자열 확인
# print("\n🔍 [빈 문자열 개수]")
# print("▶ balance 빈 문자열:\n", (balance == '').sum())

In [18]:
balance.describe().T[['min', 'max']].sort_values('min')

,min,max
연체일수_최근,-999999.0,1.0
연체일수_B1M,-999999.0,1.0
연체일수_B2M,-999999.0,1.0
최종연체회차,-99.0,0.0
RV_평균잔액_R6M,0.0,80924.0
월중평잔_RV일시불,0.0,168751.0
월중평잔_일시불,0.0,234889.0
월중평잔,0.0,330702.0
연체잔액_CA_해외_B0M,0.0,0.0
연체잔액_CA_B0M,0.0,10314.0


- 이상치
- 연체일수_최근 , 연체일수_B1M , 연체일수_B2M 의 최솟값이 -999999 
- 위 3개는 unique() 했을 때, -999999 , 0 , 1 의 값을 가진다.

- 최종연체회차의 최솟값이 -99
- 최종연체회차는 -99와 0의 값을 가진다.

In [20]:
# -999999은 데이터가 없는 것을 뜻하니, 0으로 대체
cols = ['연체일수_최근', '연체일수_B1M', '연체일수_B2M']
balance[cols] = balance[cols].replace(-999999, 0)

In [21]:
# 최종연체회차는 -99, 0 만 존재한다.
# -99는 연체정보가 없으므로 0으로 바꿔주고
# 0은 연체 정보가 있는 고객으로 간주하여 1로 바꿔준다.
# 연체정보_유무로 파생변수를 만든다.

balance['연체정보_유무'] = (balance['최종연체회차'] != -99).astype(int)

In [25]:
cols = ['연체일수_최근', '연체일수_B1M', '연체일수_B2M', '최종연체회차', '연체정보_유무']

for col in cols:
    print(balance[col].value_counts().sort_index())

연체일수_최근
0    599200
1       800
Name: count, dtype: int64
연체일수_B1M
0    599672
1       328
Name: count, dtype: int64
연체일수_B2M
0    599699
1       301
Name: count, dtype: int64
최종연체회차
-99    398092
 0     201908
Name: count, dtype: int64
연체정보_유무
0    398092
1    201908
Name: count, dtype: int64


In [27]:
# 유일값 개수가 1개 이하인 컬럼 찾기 (전부 0이거나 전부 NaN)
constant_cols = [col for col in balance.columns if balance[col].nunique(dropna=False) <= 1]

print("🧊 모든 값이 동일한 컬럼 (전부 0이거나 한 값만 있음):")
print(constant_cols)

🧊 모든 값이 동일한 컬럼 (전부 0이거나 한 값만 있음):
['카드론잔액_최종경과월', '연체잔액_대환론_B0M', '매각잔액_B1M', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_해외_B0M', '연체잔액_할부_해외_B0M', '연체잔액_CA_해외_B0M']


In [29]:
# constant_cols에 있는 컬럼들은 모두 0의 값으로만 이루어져 있으므로,
# 컬럼을 삭제한다.
balance.drop(columns=constant_cols, inplace=True)

In [31]:
# csv너무 느려서 parquet으로 다시 . .
balance.to_parquet(f'{drive_folder}test_잔액정보_통합_전처리1.parquet', index=False)

In [9]:
# 잔액정보 (모든 월 통합) 통합 parquet
balance_eda = pd.read_parquet(f'{drive_folder}test_잔액정보_통합_전처리1.parquet')
balance_eda

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,...,평잔_일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M,연체정보_유무
0,201807,TEST_00000,4802,0,0,0,0,0,6412,0,...,2933,384,0,0,0,0,0,0,0,0
1,201807,TEST_00001,1201,8628,5448,0,0,24299,1010,8156,...,657,0,0,0,7979,0,1530,0,31707,0
2,201807,TEST_00002,3046,6144,0,0,0,0,4407,8033,...,6253,0,0,0,12329,0,0,0,0,1
3,201807,TEST_00003,768,945,0,0,0,0,1107,674,...,819,0,0,0,468,0,0,0,0,0
4,201807,TEST_00004,637,2558,0,0,0,0,1072,3690,...,979,0,0,0,2414,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
599996,201812,TEST_99996,191,0,0,0,0,0,260,0,...,137,0,0,0,0,0,0,0,0,1
599997,201812,TEST_99997,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599998,201812,TEST_99998,17916,2786,0,0,0,0,16911,2081,...,18108,240,0,0,1830,0,0,0,0,0
